In [ ]:
#BAIXAR BIBLIOTECAS
!pip install matplotlib
!pip install seaborn
!pip install scikit-learn
!pip install plotly

#IMPORTAR BIBLIOTECA
#para trabalhar com base de dados
import pandas as pd
from pandas import Series
from datetime import datetime
from dateutil.parser import parse
from datetime import date, timedelta, datetime
from pandas import DataFrame

#para trabalhar com gráficos
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt
plt.style.use('ggplot')
from matplotlib.pyplot import rcParams
rcParams['figure.figsize']=15,6

#para trabalhar com análises estatísticas
from sklearn import metrics
from sklearn.metrics import mean_squared_error, mean_absolute_error
import math 

In [ ]:
#importar base de dados
#cada base de dados é referente a uma loja
dateparse = lambda dates: pd.datetime.strptime(dates, '%d/%m/%Y')
Loja_1 = pd.read_csv('Loja_1.csv', parse_dates=['DATA'],date_parser=dateparse)
display(Loja_1)

In [ ]:
#especificar produto
#escolher um produto dentro da loja para trabalhar
ProdutoGeral = Loja_1.loc[Loja_1['PRODUTO']=='A1']
ProdutoGeral = ProdutoGeral.drop(['PRODUTO'], axis=1)
#display(ProdutoGeral)
print(ProdutoGeral.info())

#plotar serie temporal
graf_ts = px.line(ProdutoGeral, x='DATA', y='QTDE_VDA')
display(graf_ts)

In [ ]:
#definir teste e treino
Produto_teste = ProdutoGeral[:200]
Produto = ProdutoGeral[201:]

#Loja_1: 347 dias no total
#Loja_2: 355 dias no total
#Loja_3: 348 dias no total

In [ ]:
#calcular médias de venda de acordo com as variáveis

#media geral
media = Produto_teste['QTDE_VDA'].mean()
display(media)

#media dia da semana (1 SEGUNDA, 7 DOMINGO)
media_semana = Produto_teste[['QTDE_VDA', 'DIA_SEMANA']].groupby(['DIA_SEMANA']).mean()
display(media_semana)

#media dia do mês
media_mes = Produto_teste[['QTDE_VDA', 'DIA_MES']].groupby(['DIA_MES']).mean()
media_mes.plot()

#media oferta
media_oferta = Produto_teste[['QTDE_VDA','OFERTA']].groupby(['OFERTA']).mean()
display(media_oferta)

In [ ]:
#calcular média combinando as variáveis de dia com a oferta

#media dia da semana (1 SEGUNDA, 7 DOMINGO)
oferta_semana = Produto_teste[['QTDE_VDA', 'DIA_SEMANA','OFERTA']].groupby(['DIA_SEMANA','OFERTA']).mean()
display(oferta_semana)

#media oferta
oferta_mes = Produto_teste[['QTDE_VDA', 'DIA_MES','OFERTA']].groupby(['DIA_MES','OFERTA']).mean()
display(oferta_mes)

In [ ]:
#definir multiplicador com índice de sazonalidade semanal, mensal e de oferta

media_semana_list = [dia/media for dia in media_semana['QTDE_VDA'].tolist()]
media_mes_list = [dia/media for dia in media_mes['QTDE_VDA'].tolist()]
media_oferta_list = [oferta/media for oferta in media_oferta['QTDE_VDA'].tolist()]
#print(media_semana_list)
#print(media_mes_list)
#print(media_oferta_list)

In [ ]:
#criar coluna de Média Móvel
Produto['MM'] = Produto.QTDE_VDA.rolling(60).mean() 

#criar coluna com Multiplicador Semanal
Produto['MULT_S'] = 'x'

#criar coluna com Multiplicador Mensal
Produto['MULT_M'] = 'x'

#criar coluna com Multiplicador Oferta
Produto['MULT_O'] = 'x'

#display(Produto)

In [ ]:
#criar uma coluna que aceite o multiplicador com base no dia da semana e dia do mês

for i, j in Produto.iterrows():
    diaSemana = j['DIA_SEMANA']
    Produto.loc[i, 'MULT_S'] = media_semana_list[diaSemana-1]
    
for i, j in Produto.iterrows():
    diaMes = j['DIA_MES']
    Produto.loc[i, 'MULT_M'] = media_mes_list[diaMes-1]

for i, j in Produto.iterrows():
    Oferta = j['OFERTA']
    Produto.loc[i, 'MULT_O'] = media_oferta_list[Oferta]

In [ ]:
#transformar as colunas criadas em colunas numéricas
Produto['MULT_S'] = Produto['MULT_S'].astype(float)
Produto['MULT_M'] = Produto['MULT_M'].astype(float)
Produto['MULT_O'] = Produto['MULT_O'].astype(float)

#apagar as linhas com células vazias (proveniente dos primeiros 60 dias usados para calcular a média móvel)
Produto = Produto.dropna(axis = 0)
display(Produto)

In [ ]:
#somar colunas MULT_S e MULT_M e subtrair 1
Produto['SUM1'] = Produto['MULT_S'] + Produto['MULT_M'] - 1
Produto['SUM2'] = Produto['MULT_S'] + Produto['MULT_O'] - 1
Produto['SUM3'] = Produto['MULT_O'] + Produto['MULT_M'] - 1

In [ ]:
#criar coluna de previsão que observe a média móvel e o multiplicativo da semana
Produto['PREVISTO_1'] = Produto['MM'].mul(Produto['MULT_S'])

#criar coluna de previsão que observe a média móvel e o multiplicativo do mês
Produto['PREVISTO_2'] = Produto['MM'].mul(Produto['MULT_M'])

#criar coluna de previsão que observe a média móvel e o multiplicativo da semana e do mês (MULTIPLICATIVO)
Produto['PREVISTO_3'] = Produto['MM'].mul(Produto['MULT_S']).mul(Produto['MULT_M'])

#criar coluna de previsão que observe a média móvel e o multiplicativo da semana e do mês (ADITIVO)
Produto['PREVISTO_4'] = Produto['MM'].mul(Produto['SUM1'])

#criar coluna de previsão que observe a média móvel e o multiplicativo de oferta e da semana (MULTIPLICATIVO)
Produto['PREVISTO_5'] = Produto['MM'].mul(Produto['MULT_O']).mul(Produto['MULT_S'])

#criar coluna de previsão que observe a média móvel e o multiplicativo de oferta e do mês (MULTIPLICATIVO)
Produto['PREVISTO_6'] = Produto['MM'].mul(Produto['MULT_O']).mul(Produto['MULT_M'])

#criar coluna de previsão que observe a média móvel e o multiplicativo de oferta e da semana (ADITIVO)
Produto['PREVISTO_7'] = Produto['MM'].mul(Produto['SUM2'])

#criar coluna de previsão que observe a média móvel e o multiplicativo de oferta e do mês (ADITIVO)
Produto['PREVISTO_8'] = Produto['MM'].mul(Produto['SUM3'])

#criar coluna de previsão que observe a média móvel e o multiplicativo de oferta
Produto['PREVISTO_9'] = Produto['MM'].mul(Produto['MULT_O'])

In [ ]:
#transformar as colunas criadas em colunas numéricas
Produto['PREVISTO_1'] = Produto['PREVISTO_1'].astype(float)
Produto['PREVISTO_2'] = Produto['PREVISTO_2'].astype(float)
Produto['PREVISTO_3'] = Produto['PREVISTO_3'].astype(float)
Produto['PREVISTO_4'] = Produto['PREVISTO_4'].astype(float)
Produto['PREVISTO_5'] = Produto['PREVISTO_5'].astype(float)
Produto['PREVISTO_6'] = Produto['PREVISTO_6'].astype(float)
Produto['PREVISTO_7'] = Produto['PREVISTO_7'].astype(float)
Produto['PREVISTO_8'] = Produto['PREVISTO_8'].astype(float)
Produto['PREVISTO_9'] = Produto['PREVISTO_9'].astype(float)
#Produto.info()

#apagar as linhas com células vazias (proveniente dos primeiros 60 dias usados para calcular a média móvel)
Produto = Produto.dropna(axis = 0)
#display(Produto)

In [ ]:
#plotar gráfico  
plt.figure(figsize = ( 20, 8)) 
sns.lineplot(x = 'DATA', 
             y = 'QTDE_VDA', 
             data = Produto, 
             label = 'Serie Temporal',
             marker = 'o',
             color = 'red') 
sns.lineplot(x = 'DATA', 
             y = 'MM', 
             data = Produto, 
             label = 'Média Móvel',
             marker = 'o',
             color = 'blue')
sns.lineplot(x = 'DATA',
            y = 'PREVISTO_1',
            data = Produto,
            label = 'Mult. Semanal',
            marker = 'o',
            color = 'green')
sns.lineplot(x = 'DATA',
            y = 'PREVISTO_2',
            data = Produto,
            label = 'Mult. Mensal',
            marker = 'o',
            color = 'purple')
sns.lineplot(x = 'DATA',
            y = 'PREVISTO_3',
            data = Produto,
            label = 'Mult. S e M',
            marker = 'o',
            color = 'black')
sns.lineplot(x = 'DATA',
            y = 'PREVISTO_4',
            data = Produto,
            label = 'Mult. S + M',
            marker = 'o',
            color = 'orange')
sns.lineplot(x = 'DATA', 
             y = 'PREVISTO_5', 
             data = Produto, 
             label = 'Mult. S e O',
             marker = 'o',
             color = 'gray')
sns.lineplot(x = 'DATA',
            y = 'PREVISTO_6',
            data = Produto,
            label = 'Mult. M e O',
            marker = 'o',
            color = 'pink')
sns.lineplot(x = 'DATA',
            y = 'PREVISTO_7',
            data = Produto,
            label = 'Mult. S + O',
            marker = 'o',
            color = 'yellow')
sns.lineplot(x = 'DATA',
            y = 'PREVISTO_8',
            data = Produto,
            label = 'Mult. M + O',
            marker = 'o',
            color = 'brown')
sns.lineplot(x = 'DATA',
            y = 'PREVISTO_9',
            data = Produto,
            label = 'Mult. Oferta',
            marker = 'o',
            color = 'green')
plt.show()

In [ ]:
#ACURÁCIA DO MODELO
test = Produto['QTDE_VDA']
future_MM = Produto['MM']
future_1 = Produto['PREVISTO_1']
future_2 = Produto['PREVISTO_2']
future_3 = Produto['PREVISTO_3']
future_4 = Produto['PREVISTO_4']
future_5 = Produto['PREVISTO_5']
future_6 = Produto['PREVISTO_6']
future_7 = Produto['PREVISTO_7']
future_8 = Produto['PREVISTO_8']
future_9 = Produto['PREVISTO_9']

rmse = math.sqrt(mean_squared_error(test, future_MM))
print('Test RMSE: %.3f' % rmse)

rmse = math.sqrt(mean_squared_error(test, future_1))
print('Test RMSE: %.3f' % rmse)

rmse = math.sqrt(mean_squared_error(test, future_2))
print('Test RMSE: %.3f' % rmse)

rmse = math.sqrt(mean_squared_error(test, future_3))
print('Test RMSE: %.3f' % rmse)

rmse = math.sqrt(mean_squared_error(test, future_4))
print('Test RMSE: %.3f' % rmse)

rmse = math.sqrt(mean_squared_error(test, future_5))
print('Test RMSE: %.3f' % rmse)

rmse = math.sqrt(mean_squared_error(test, future_6))
print('Test RMSE: %.3f' % rmse)

rmse = math.sqrt(mean_squared_error(test, future_7))
print('Test RMSE: %.3f' % rmse)

rmse = math.sqrt(mean_squared_error(test, future_8))
print('Test RMSE: %.3f' % rmse)

rmse = math.sqrt(mean_squared_error(test, future_9))
print('Test RMSE: %.3f' % rmse)